本文大部分内容来自：
- http://hyperopt.github.io/hyperopt/
- https://www.jiqizhixin.com/articles/2017-08-21-7

## 动机
深度网络的超参数往往会比较多，如何选取合适的值，则成为一个很困扰的问题。比如，这里是卷积神经元中的一些经典的超参数，究竟应该选哪个，希望这篇文章能提供解决的方法：
![hyperparam](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/hyperParameter.png)

## 几种可能的方法
- 启动网格搜索grid search，尝试检查每种可能的参数组合，当有一种组合优化了你的标准时，就停止搜索。这种方法一般比较耗时。它有个致命缺点，在高纬度会发生维度灾难
- 可以使用随机搜索，这种方法可以随机检查超参数的空间，但速度更快而且大多时候也更好
- 贝叶斯优化--我们为超参数设置一个先决条件，然后在观察不同实验的同时逐步更新他，这让我们可以更合理的拟合超参数空间，从而更好的找到最小值。它的本质其实是一种回归模型，即利用回归模型预测的函数值来选择下一个搜索点。贝叶斯优化有两个主要的要素：目标函数（objective function）和代理模型（surrogate model）。

In [ ]:
贝叶斯优化算法简化
选取m个点x1,..xm作为prior，假设他们服从多变量高斯分布
for t=1,2...n do
  最大化AF求得下一个采样点x0
  采样目标函数位于x0处的值y0
  假设x0,x1....xm服从多变量高斯分布
# 其中AF是收获函数

## Hyperopt

Hyperopt库为python中的模型选择和参数优化提供了算法和并行方案。机器学习常见的模型有KNN,SVM，PCA，决策树，GBDT等一系列的算法，但是在实际应用中，我们需要选取合适的模型，并对模型调参，得到一组合适的参数。尤其是在模型的调参阶段，需要花费大量的时间和精力，却又效率低下。但是我们可以换一个角度来看待这个问题，模型的选取，以及模型中需要调节的参数，可以看做是一组变量，模型的质量标准（比如正确率，AUC）等等可以看做是目标函数，这个问题就是超参数的优化的问题。我们可以使用搜索算法来解决。

Hyperopt提供了一个优化接口，这个接口接受一个评估函数和参数空间，能计算出参数空间内的一个点的损失函数值。用户还要指定空间内参数的分布情况。<br> 
Hyheropt四个重要的因素：
- 指定需要最小化的函数 
- 指定进行搜索的空间
- 采样的数据集(trails database)（可选）(使用mongodb可以实现并行)
- 搜索的算法（可选）

### 定义函数
在定义函数的时候，主要要考虑一下三个方面：
- 除了函数返回的数值外，是否要保持其他有用的统计数据
- 优化算法，是否要用到除了函数返回值之外的其他数据
- Do you want to communicate between parallel processes?(e.g. other workers, or the minimization algorithm???)

##### 最简单的情况
hyperopt优化算法和目标方程之间，最简单的一种协议就是，目标方程使用搜索空间中的一个点作为输入，输出其对应的loss

In [ ]:
# 安装hyperopt
pip install hyperopt

In [5]:
from hyperopt import fmin, tpe, hp
best = fmin(fn = lambda x: x ** 2,
           space = hp.uniform('x', -10, 10), 
           algo = tpe.suggest,
           max_evals = 100)
print best

TypeError: 'generator' object has no attribute '__getitem__'

不知道为什么会这样，本地运行是没有问题的。输出{'x': -0.008370279913737733}<br>
这种方法的缺点是：
- 没有办法返回额外的信息
- 没办法实现和优化方法之间的互动

##### 通过Trials对象，增添额外的信息
当你的目标函数比较复杂，要求花费比较多的时间进行运行的时候，你可能会想，除了目标函数的输出之外，存储其他有用的统计信息。在这种情况下，我们可以改写目标函数，使得他可以返回dict类型的数据。并在这个词典类型中包含你所需要的统计信息。但需要注意的是，这个词典类型应该满足一定的条件。比如，对于mongodb来说，这个词典类型数据就必须符合JSON文档要求。<br>
当目标函数返回的是词典类型的时候，fmin会获取其中的一些特定的key-value对，并把这个值传递给优化算法。两个指定的key-value对是：
- status: 对应hyperopt.STATUS_STRINGS，比如ok代表成功完成，fail代表函数未定义？？？？
- loss: 要求进行最小化的，浮点型的函数值。当status是ok的时候，这个值必须存在。

除此之外，还有一些可选的、常用的、设置好的key：
- attachments: 对应的value是一个词典类，其中的项，通常是短字符串和长字符串的对，一般短字符串代表一个文件的名称，而长字符串代表其对应的内容。这些数据在定义的时候，就会被存入内存，从而避免每次从数据库中读取数据的时候，都要重复读取。另外，像mongodb这些数据库对其中value的大小都是有限定的，也就是说，当你的项是mb大小的时候，你就不得不把他们弄成attachment了。(不明白，把他放入数据库的record中，是因为要不对的进行改变，尝试，但是如果用attachment的话，不是就固定是一个值了吗，那么还有什么意义。直接在目标函数中进行处理不就可以了吗？？)
- loss_variance: 浮点类型，代表概率目标函数的不确定性。（这个真得自己算吗？？，扔这有什么意义。）
- true_loss: When doing hyper-parameter optimization, if you store the generalization error of your model with this name, then you sometimes get spiffier output from the build-in plotting routines.(就是指generalization error吗)
- true_loss_variance: float- the uncertainty in the generalization error

In [ ]:
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(x):
    return {'loss': x ** 2,
           'status': STATUS_OK}

best = fmin(objective,
            space = hp.uniform('x', -10, 10),
            algo = tpe.suggest,
            max_evalas = 100)

##### Trail对象

In [ ]:
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def objective(x):
    return{
        'loss': x**2,
        'status': STATUS_OK,
        # -- store other results like this
        'eval_time':time:time()
        'other_stuff':{'type':None, 'value':[0, 1, 2]}
        # -- attachments are handled differently
        'attachemnts':{
            'time_module': pickle.dumps(time.time)
        }
    }

trials = Trials()
best = fime(objective,
            space = hp.uniform('x', -10, 10),
            algo = tpe.suggest,
            max_evals = 100,
            trials = trials)
print best

通过加入trials，可以实现在实验的过程中，访问目标函数的返回值:
- trials.trials: a list of dictionaries representing everything about the search
- trials.results: a list of dictionaries returned by 'objective' during the search
- trials.losses(): a list of losses(float for each 'ok' trial)
- trials.statuses(): a list of status strings

The trials object can be saved, passed on to the built-in plotting routines, or analyzed with your own custom code.
##### The Ctrl Object for Realtime Communication with MongoDB
?????

## 定义搜索空间
一个搜索空间由一个或多个相互嵌套的函数表达组合而成。（这些函数表达，包括概率表达式）

In [ ]:
from hyperopt import hp
space = hp.choice('a',
                 [
                     ('case1', 1 + hp.lognormal('c1', 0, 1)),
                     ('case2', hp.uniform('c2', -10, 10))
                 ])

你可以哦谈过pyll来，获取这个空间中的一个实例：

In [ ]:
import hyperopt.pyll.stochastic
print hyperopt.pyll.stochastic.sample(space)

#### 常用的参数表达式
- hp.choice(label, options): options可以是一个list也可以是一个tuple。这个函数返回一个选项。
- hp.randint(label, upper): 返回[0, upper)中的一个整数。
- hp.uniform(label, low, high): Returns a value uniformly between low and high. 
    - When optimizing, this variable is constrained to a two-sided interval.????
- hp.quniform(label, low, high, q): 返回round(uniform(low, high)/q)*q。
    - Suitable for a discrete value with respect to which the objective is still somewhat "smooth", but which should be bounded both above and below。
- hp.loguniform(label, low, high): 返回exp(uniform(low, high))。 
    - When optimizing, this variable is constrained to the interval [exp(low), wxp(high)]
- hp.qloguniform: 返回round(exp(uniform(low, high))/q)*q。
    - Suitable for a discrete variable with respect to which the objective is "smooth" and gets smoother with the size of the value, but which should be bounded both above and below.
- hp.normal(label, mu, sigma)
- hp.qnormal(label, mu , sigma , q): 返回rount(normal(mu, sigma)/q)*q。
    - Suitable for a discrete variable  that probably takes a value around mu, but is fundamentally unbounded.
- hp.lognormal(label, mu, sigma): 返回exp(normal(mu, sigma))
- hp.qlognormal(label, mu, sigma, q): 返回round(exp(normal(mu, sigma))/q)*q

#### 一个关于搜索空间的例子

In [ ]:
from hyperopt import hp
space = hp.choice('classifier_type',[
    {
        'type': 'naive_bayes',
    },
    {
        'type': 'svm',
        'C': hp.lognormal('sum_C', 0, 1),
        'kernel': hp.choice('svm_kernel', [
            {'ktype': 'linear'},
            {'ktype': 'RBF', 'width': hp.lognormal('svm_rbf_width', 0, 1)}
        ])
    },
    {
        'type': 'dtree',
        'criterion': hp.choice('dtree_criterion', ['gini', 'entropy']),
        'max_depth': hp.choice('dtree_max_depth',
                               [None, hp.qlognormal('dtree_max_depth_int', 3, 1, 1)]),
        'min_samples_split': hp.qlognormal('dtree_min_samples_split', 2, 1, 1),
    },
])

#### Adding Non-Stochastic Expressions with pyll
???
#### Adding new Kinds of Hyperparameter
???

## 其他补充

搜索算法本身也有内置的参数决定如何去优化目标函数，我们可以指定搜索算法的参数，比如针对TPE，指定jobs：

In [ ]:
from functools import partial
from hyperopt import hp, fmin, tpe
algo = partial(tpe.suggest, n_startup_jobs=10)
best = fmin(q, space, algo=algo)
print space_eval(space, best)

关于参数空间的设置，比如优化函数q，输入fmin(q,space=hp.uniform(‘a’,0,1)).hp.uniform函数的第一个参数是标签，每个超参数在参数空间内必须具有独一无二的标签。hp.uniform指定了参数的分布。其他的参数分布比如 <br>
hp.choice返回一个选项，选项可以是list或者tuple.options可以是嵌套的表达式，用于组成条件参数。 <br>
hp.pchoice(label,p_options)以一定的概率返回一个p_options的一个选项。这个选项使得函数在搜索过程中对每个选项的可能性不均匀。 <br>
hp.uniform(label,low,high)参数在low和high之间均匀分布。 <br>
hp.quniform(label,low,high,q),参数的取值是round(uniform(low,high)/q)*q，适用于那些离散的取值。 <br>
hp.loguniform(label,low,high)绘制exp(uniform(low,high)),变量的取值范围是[exp(low),exp(high)] <br>
hp.randint(label,upper) 返回一个在[0,upper)前闭后开的区间内的随机整数。 <br>
搜索空间可以含有list和dictionary.<br>

In [ ]:
from hyperopt import hp
list_space = [
hp.uniform(’a’, 0, 1),
hp.loguniform(’b’, 0, 1)]
tuple_space = (
hp.uniform(’a’, 0, 1),
hp.loguniform(’b’, 0, 1))
dict_space = {
’a’: hp.uniform(’a’, 0, 1),
’b’: hp.loguniform(’b’, 0, 1)}

使用sample函数从参数空间内采样：

In [ ]:
from hyperopt.pyll.stochasti import sample
print sample(list_space)
# => [0.13, .235]
print sample(nested_space)
# => [[{’case’: 1, ’a’, 0.12‘}, {’case’: 2, ’b’: 2.3}],
# ’extra_literal_string’,
# 3]

在参数空间内使用函数：

In [ ]:
from hyperopt.pyll import scope
def foo(x):
return str(x) ∗ 3
expr_space = {
’a’: 1 + hp.uniform(’a’, 0, 1),
’b’: scope.minimum(hp.loguniform(’b’, 0, 1), 10),
’c’: scope.call(foo, args=(hp.randint(’c’, 5),)),
}